In [70]:
import pandas as pd
from rdkit import Chem
import numpy as np
from rdkit.Chem import PandasTools
from rdkit import RDConfig
PandasTools.RenderImagesInAllDataFrames(images=True)
import os

In [71]:
lines=[]
with open("patents_ochem_enamine_bradley_begstrom_training_.csv","r") as f:
    l = f.readline()
    while l:
        l = l.strip().replace('","',"'\t'").replace('"',"''").replace("'\t'",'","')
        if l.startswith("''"):
            l='"'+l[2:]
        if l.endswith("''"):
            l=l[:-2]+'"'
        l=l+"\n"
        
        lines.append(l)
        l = f.readline()
    
with open("patents_ochem_enamine_bradley_begstrom_training_edit.csv","w+") as f:
    f.writelines(lines)

In [111]:
df = pd.read_csv("patents_ochem_enamine_bradley_begstrom_training_edit.csv")

In [107]:
from rdkit.Chem import MolFromSmiles
def _mfs(s):
    return MolFromSmiles(s)
df["mol"]=df["smiles"].apply(_mfs)

In [108]:
from rdkit.Chem import MolToSmiles
def _mts(m):
    try:
        return MolToSmiles(m)
    except:
        return ""
df["new_SMILES"]=df["mol"].apply(_mts)
df.drop(df.index[df["new_SMILES"]==""],inplace=True)

In [112]:
df.drop(["NAME.1","UNIT {Melting Point}","CASRN","SMILES","mol"],axis=1,errors="ignore",inplace=True)
df.rename({"Melting Point {measured, converted}":"mpC","new_SMILES":"smiles","mpc":"mpC"},axis=1,inplace=True)

In [121]:
for smiles in df["smiles"][df["smiles"].duplicated()]:
    _d = df[df["smiles"]==smiles]
    _mean= _d.mpC.mean()
    if np.abs(_d.mpC-_mean).max() > 5:
        df.drop(_d.index,inplace=True)
    else:
        _di=_d.index
        df.loc[_d.index[0],"mpC"]=_mean
        df.drop(_d.index[1:],inplace=True)
df.reset_index(drop=True, inplace=True)

In [125]:
df.to_csv("patents_ochem_enamine_bradley_begstrom_training_edit.csv",index=False)